## Streaming from Twitter at localhost with Pyspark

In [ ]:
#This is the regular Jupyter notebook, not launched at pyspark

In [ ]:
import findspark

In [ ]:
findspark.init('/Users/binggangliu/spark')

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [ ]:
sc = SparkContext()

In [ ]:
ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)

#### Streaming data from the localhost that listens to Twitter API

In [ ]:
socket_stream = ssc.socketTextStream("127.0.0.1", 5556)

In [ ]:
lines = socket_stream.window(20)

In [ ]:
from collections import namedtuple
fields = ("tag", "count")
Tweet = namedtuple('Tweet', fields)

In [ ]:
# use parenthesis for multiple lines or use \.
(lines.flatMap(lambda text: text.split(" "))
  .filter(lambda word: word.lower().startswith("#"))
  .map(lambda word: (word.lower(), 1))
  .reduceByKey(lambda a, b: a+b)
  .map(lambda rec: Tweet(rec[0], rec[1]))
  .foreachRDD(lambda rdd: rdd.toDF().sort(desc("count")).limit(5).registerTempTable("tweets"))
)

#### Run pyspark_twitter_read.py at this point

In [ ]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
%matplotlib inline

In [ ]:
ssc.start()

#### visualize top count hashtags

In [ ]:
ct = 0
while ct < 20:
    time.sleep(3)
    top_5_tweets = sqlContext.sql('Select tag, count from tweets')
    top_5_df = top_5_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure(figsize = (10,8))
    sns.barplot(x="count", y="tag", data=top_5_df)
    plt.show()
    ct=ct+1

In [ ]:
ssc.stop()